# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:

import os

openai_api_key = os.environ['OPENAI_MODEL_NAME'] = os.getenv('OPENAI_API_KEY')

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
    # allow_delegation=True as default
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools
- Import CrewAI tools

In [ ]:
%pip install crewai_tools==0.1.6

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

  Using cached crewai_tools-0.1.6-py3-none-any.whl (47 kB)
  Attempting uninstall: crewai-tools
    Found existing installation: crewai-tools 0.2.6
    Uninstalling crewai-tools-0.2.6:
      Successfully uninstalled crewai-tools-0.2.6
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\brics\Documents\git_repos\Interactive_RAG_e-book\venv\Scripts\python.exe -m pip install --upgrade pip' command.


### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```
- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [21]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.heartmath.com/"
)
docs_scrape_tool

ScrapeWebsiteTool(name='Read website content', description="Read website content() - A tool that can be used to read https://www.heartmath.com/'s content.", args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.FixedScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x000001D104E57400>, website_url='https://www.heartmath.com/', cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Language': 'en-US,en;q=0.9', 'Referer': 'https://www.google.com/', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'Accept-Encoding': 'gzip, deflate, br'})

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [10]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)


### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [22]:
inputs = {
    "customer": "HeartMath",
    "person": "Roberta",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: HeartMath just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Roberta from HeartMath is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to make sure I provide Roberta from HeartMath with the best support possible by thoroughly addressing her question on setting up a Crew and adding memory to it. I must use the "Read website content" tool to gather the necessary information.

Action:
Read website content

Action Input:
{"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"}
 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crewAI
 Assembling and Activating 

- Display the final result as Markdown.

In [23]:
from IPython.display import Markdown
Markdown(result)

Dear Roberta,

Thank you for reaching out with your inquiry about setting up a Crew and adding memory to it. I'm here to provide you with comprehensive guidance on how to achieve this effectively using CrewAI.

To add memory to your Crew in CrewAI, you need to follow a series of steps outlined in the documentation provided on our website. Here's a detailed breakdown of the process:

1. **Step 0: Installation**: Start by installing CrewAI and any necessary packages for your project. Make sure you have Python version >=3.10 and <=3.13. You can use the following commands:
   ```
   pip install crewai
   pip install 'crewai[tools]'
   ```

2. **Step 1: Assemble Your Agents**: Define your agents with distinct roles, backstories, and enhanced capabilities like memory usage. You can create agents with memory by specifying the `memory=True` parameter in the agent creation process. Here's an example code snippet:
   ```
   # Creating a senior researcher agent with memory
   researcher = Agent(
     role='Senior Researcher',
     goal='Uncover groundbreaking technologies in {topic}',
     verbose=True,
     memory=True,
     backstory="Driven by curiosity..."
   )
   ```

3. **Step 3: Form the Crew**: Combine your agents into a crew and set the workflow process they'll follow to accomplish tasks. You can enable memory for the crew by specifying `memory=True` in the crew creation process. Here's an example code snippet:
   ```
   # Forming the crew with memory
   crew = Crew(
     agents=[researcher, writer],
     tasks=[research_task, write_task],
     process=Process.sequential,
     memory=True,
     cache=True,
     max_rpm=100,
     share_crew=True
   )
   ```

4. **Step 4: Kick It Off**: Initiate the process with your crew ready. Input variables will be interpolated into the agents and tasks for a personalized approach. Here's an example code snippet:
   ```
   result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
   print(result)
   ```

By following these steps, you'll be able to set up a Crew in CrewAI with memory capabilities, allowing your agents to store and access information for improved performance and efficiency.

If you need further assistance or have any other questions, feel free to reach out. We're here to help you succeed with CrewAI!

Best regards,
[Your Name]
Senior Support Representative
crewAI